**VMSP algorithm to find maximal sequence patterns**

VMSP (Vertical maximal sequence patterns) is, well a pattern mining algorithm to find _maximal_ sequential patterns from a dataset containing event sequences. The original paper is present at [link](https://www.philippe-fournier-viger.com/spmf/VMSP_maximal_sequential_patterns_2014.pdf).

Some glossary:

**Event sequence** - an ordered list of event objects. Each event specifies an event type, though could also have other attributes. 

In [1]:
# tennis shot by shot data
data = [
    [
        {"typ": "serve", "player": 1, "winner": False},
        {"typ": "backhand", "player": 2, "winner": False},
        {"typ": "backhand", "player": 1, "winner": True},
    ],
    [
        {"typ": "serve", "player": 1, "winner": True},
    ],
]

**Subsequence** - A subset of the events in a sequence; doesn't need to be contiguous but retains the order in the original sequence. 

**Pattern** - also specified as a list of events. If a sequence has a subsequence matching each event in the pattern, it is a match. 

**Support** - number of sequences in the dataset that match against a pattern. 

**Maximal pattern** - a pattern is maximal if no other pattern obtained by appending events to it has a support greater than the threshold minimum support.

The core algorithm is pretty straightforward, and works by extending the current set of frequent patterns with more events. A number of heuristics are used to prune alternatives that won't result in maximal patterns. 

We implement it first with regular python objects. Sequences are list of strings, and the dataset is a list of sequences. 

NOTE: We deviate somewhat from the full algorithm, in that the general algorithm works with `itemsets`, which are sets of events instead of a single event at each index of the sequence. The sequences looks like `{A}-{A,B}-{C}-{A,B}` instead of `A-B-A-C-B-B`. We stick to single event at an index sequences. 

## regular python

In [2]:
dataset = [
    ["a", "b", "c", "d"],
    ["b", "c", "d", "e"],
    ["c", "d", "e", "f"],
    ["d", "e", "f", "g"],
    ["e", "f", "g", "h"],
]

In [3]:
def match_seq(seq: list[str], pat: list[str]) -> bool:
    """find out if the pattern matches the sequence"""
    if len(pat) == 0:
        return True
    for i in range(len(seq)):
        if seq[i] == pat[0]:
            return match_seq(seq[i + 1 :], pat[1:])
    return False


def calc_support(pat: list[str]) -> int:
    """find the num of sequences that match this pattern"""
    count = 0
    for seq in dataset:
        if match_seq(seq, pat):
            count += 1
    return count


def search(
    result: list[list[str]], pat: list[str], extend_set: set[str], min_sup: int
) -> None:
    """try to extend the input pattern and check if it is maximal"""
    next_extend_set: set[str] = set()
    for evt in extend_set:
        if calc_support([*pat, evt]) >= min_sup:
            next_extend_set.add(evt)

    # recurse on the next set of candidate patterns
    for evt in next_extend_set:
        search(result, [*pat, evt], next_extend_set, min_sup)

    # if no extended patterns are frequent, the current pattern might be maximal
    if len(next_extend_set) == 0:
        for pat_i in result:
            # we gotta check if any existing pattern is a super pattern to this one
            if len(pat_i) > len(pat) and match_seq(pat_i, pat):
                return

        result.append(pat)
        # also cut out any existing patterns that are sub patterns to this one
        cut_indices = []
        for i, pat_i in enumerate(result):
            if len(pat_i) < len(pat) and match_seq(pat, pat_i):
                cut_indices.append(i)
        for i in cut_indices:
            result.pop(i)


def enumerate_patterns(min_sup: int):
    """find all patterns that match at least `min_sup` number of sequences"""
    # find the most frequent events, cut out the ones below the threshold
    event_counts = dict()
    for seq in dataset:
        for evt in seq:
            if evt in event_counts:
                event_counts[evt] += 1
            else:
                event_counts[evt] = 1

    result: list[list[str]] = []
    extend_set = set(e for e, c in event_counts.items() if c >= min_sup)
    for evt in extend_set:
        search(result, [evt], extend_set, min_sup)

    return result

For ex, to find patterns with minimum support of 3.

In [4]:
enumerate_patterns(3)

[['d', 'e'], ['e', 'f'], ['c', 'd']]

We expect to see some interesting patterns from mining, though not too many. Further, longer patterns likely convey more information than a prefix of them. This makes maximal patterns a reasonable set to work with. 

### Optimizing the search

The problem size grows with:
* dataset with large num of sequences
* sequences with long list of events

The primitives that get called a lot:
1. calculating support for pattern (calls match-sequence for each sequence in the dataset)
2. match-sequence to check if one pattern is a super/sub pattern of another

The original paper uses a bunch of heuristics to reduce time spent doing both. 

* For 1, create a data structure called `id-list` for each event type, that records all instances of the event across all sequences as `(sequence-id, index-of-event-in-sequence)` tuples. Support for any pattern can now be calculated by _joining_ these id-lists for the events in it.
* For 2, use a heap like container to store maximal patterns and cut down on these function calls. 

What we implement:
* I like databases. Duckdb provides really quick range joins, which we can use to implement 1. 
* I don't like fancy data structures. For large datasets, 2 might not be that big a factor anyway since 1 probably consumes most cycles. 

## Foursquare dataset

We use the foursquare NYC check-ins dataset to test the algorithm (inspired by the MAQUI [paper](https://www.zcliu.org/maqui/)). We want to look at the sequences of places a user visits within a full day.

In [5]:
import duckdb as db
import numpy as np
import pyarrow as pa
import pyarrow.parquet

In [6]:
ds = pa.parquet.read_table("../datasets/foursquare_nyc.parquet")
ds.schema

userId: int64
venueId: string
venueCategoryId: string
venueCategory: string
latitude: double
longitude: double
timezoneOffset: int64
utcTimestamp: timestamp[ms, tz=UTC]

In [7]:
# some math to get the timestamp in local timezone, so we can segment event sequences by day.
local_times = pa.array(
    ds["utcTimestamp"] + ds["timezoneOffset"].to_numpy().astype("timedelta64[m]"),
    pa.timestamp("s", tz=None),
)

# The event identifier we want to use is the `venueCategory`. Could dictionary encode it so we only have integers to work wt
venue_dict_ids = pa.compute.dictionary_encode(ds["venueCategory"]).combine_chunks()

In [8]:
venue_categories = venue_dict_ids.dictionary.tolist()
ds = (
    ds.remove_column(ds.column_names.index("utcTimestamp"))
    .append_column("localTimestamp", local_times)
    .remove_column(ds.column_names.index("venueCategoryId"))
    .append_column("venueCategoryId", venue_dict_ids.indices)
)

In [9]:
conn = db.connect()
conn.register("foursq_data", ds)

We are good to query the duckdb dataset now.

In [10]:
# ds.column_names

In [11]:
def read_category(cat_id: int) -> str:
    return venue_categories[cat_id]

In [12]:
conn.execute(
    """
CREATE OR REPLACE TABLE 
    events_binned 
AS
SELECT 
    userId, 
    "localTimestamp",
    DATE_PART('epoch', "localTimestamp") / 86400 AS dateBin,
    venueCategoryId,
    ROW_NUMBER() OVER (
        PARTITION BY 
            userId, 
            DATE_PART('epoch', "localTimestamp") / 86400
        ORDER BY 
            "localTimestamp" ASC
    ) AS seqIndex
FROM
    foursq_data
"""
)

In [13]:
read_category(2), read_category(6)

('Home (private)', 'Coffee Shop')

Lets look for sequences when users were at home and visited the coffee shop after, on some day.

In [14]:
conn.execute(
    """
WITH 
evt1 AS (
    SELECT * 
    FROM events_binned
    WHERE venueCategoryId = 2
),
evt2 AS (
    SELECT * 
    FROM events_binned
    WHERE venueCategoryId = 6
)
SELECT COUNT(*)
FROM (
    SELECT DISTINCT evt1.userId, evt1.dateBin
    FROM evt1
    INNER JOIN evt2
    ON
        evt1.userId = evt2.userId AND
        evt1.dateBin = evt2.dateBin AND
        evt1.seqIndex < evt2.seqIndex
)
"""
)
conn.fetchall()

[(270,)]

pretty neat! This could be extended to longer sequences of events but we gotta construct those queries programatically. 

NOTE: A further optimization could be to construct a separate table for each event category noting all the `seqIndex` values for it. That would be exactly the same data structure as the `id-list` in the original paper. Duckdb filters are pretty quick though, so we skip it. 

NOTE #2: If a sequence contains multiple instances of a pattern, we still count it once.

In [15]:
def assemble_support_query(pat_sequence: list[int]) -> str:
    cte_clauses = []
    for i, evt_id in enumerate(pat_sequence):
        lines = []
        lines.append(f"view_{i} AS (")
        lines.append("\t" + "SELECT t.userId, t.dateBin, min(t.seqIndex) AS seqIndex")
        lines.append("\t" + "FROM events_binned AS t")

        if i != 0:
            lines.append("\t" + f"INNER JOIN view_{i-1}")
            lines.append(
                "\t"
                + f"ON view_{i-1}.userId = t.userId AND view_{i-1}.dateBin = t.dateBin AND view_{i-1}.seqIndex < t.seqIndex"
            )

        lines.append("\t" + f"WHERE t.venueCategoryId = {pat_sequence[i]}")
        lines.append("\t" + "GROUP BY t.userId, t.dateBin")
        lines.append(")")
        cte_clauses.append("\n".join(lines))
    cte_string = "WITH\n" + ",\n".join(cte_clauses)

    select_string = f"SELECT COUNT(*) FROM view_{i}"
    return cte_string + "\n" + select_string

In [16]:
# print(assemble_support_query([2, 6, 10]))

Let's look for the most frequent event categories.

In [17]:
conn.execute(
    """
SELECT 
    venueCategoryId, COUNT(*) 
FROM events_binned 
GROUP BY venueCategoryId
ORDER BY COUNT(*) DESC
LIMIT 20
"""
)
res = conn.fetchall()
for i, count in res:
    print(f"{read_category(i)} - {count}")

Bar - 15978
Home (private) - 15382
Office - 12740
Subway - 9348
Gym / Fitness Center - 9171
Coffee Shop - 7510
Food & Drink Shop - 6596
Train Station - 6408
Park - 4804
Neighborhood - 4604
Bus Station - 4474
Deli / Bodega - 4214
Residential Building (Apartment / Condo) - 4185
Other Great Outdoors - 4134
American Restaurant - 3701
College Academic Building - 3479
Building - 3474
Medical Center - 3366
Road - 3207
Clothing Store - 2976


Bars are visited as often as homes. Reasonable. 

To find out maximal patterns, we need a min support value. The total num of sequences can be counted as

In [18]:
conn.execute(
    """SELECT COUNT(*) FROM (SELECT DISTINCT userId, dateBin FROM events_binned)"""
)
conn.fetchall()

[(93862,)]

Lets work with a min support value of 3000.

### Rewriting the search routine

In [19]:
def get_frequent_events(sup: int):
    q = """
SELECT 
    venueCategoryId, COUNT(*) AS count
FROM events_binned 
GROUP BY venueCategoryId
HAVING COUNT(*) > {sup}
""".format(
        sup=sup
    )

    conn.execute(q)
    res = conn.fetchall()
    return [row[0] for row in res]

In [20]:
def match_seq(seq: list[int], pat: list[int]) -> bool:
    """find out if the pattern matches the sequence"""
    if len(pat) == 0:
        return True
    for i in range(len(seq)):
        if seq[i] == pat[0]:
            return match_seq(seq[i + 1 :], pat[1:])
    return False


def calc_support(pat: list[int]) -> int:
    """find the num of sequences that match this pattern"""
    # we replaced the iteration over all sequences with a SQL query to count it
    assert len(pat) > 0
    conn.execute(assemble_support_query(pat))
    return conn.fetchone()[0]


def search(
    result: list[list[int]], pat: list[int], extend_set: set[int], min_sup: int
) -> None:
    """try to extend the input pattern and check if it is maximal"""
    next_extend_set: set[int] = set()
    for evt in extend_set:
        if calc_support([*pat, evt]) >= min_sup:
            next_extend_set.add(evt)

    # recurse on the next set of candidate patterns
    for evt in next_extend_set:
        search(result, [*pat, evt], next_extend_set, min_sup)

    # if no extended patterns are frequent, the current pattern might be maximal
    if len(next_extend_set) == 0:
        for pat_i in result:
            # we gotta check if any existing pattern is a super pattern to this one
            if len(pat_i) > len(pat) and match_seq(pat_i, pat):
                return

        result.append(pat)
        # also cut out any existing patterns that are sub patterns to this one
        cut_indices = []
        for i, pat_i in enumerate(result):
            if len(pat_i) < len(pat) and match_seq(pat, pat_i):
                cut_indices.append(i)
        for i in cut_indices:
            result.pop(i)


def enumerate_patterns(min_sup: int):
    """find all patterns that match at least `min_sup` number of sequences"""
    # find the most frequent events, cut out the ones below the threshold
    extend_set = set(get_frequent_events(min_sup))
    result: list[list[int]] = []
    for evt in extend_set:
        search(result, [evt], extend_set, min_sup)

    return result

In [21]:
%%time
patterns_mined = enumerate_patterns(500)
for pat in patterns_mined:
    if len(pat) > 1:
        print("--".join(read_category(x) for x in pat))

Home (private)--Bar
Home (private)--Home (private)--Home (private)
Home (private)--Office
Home (private)--Other Great Outdoors
Home (private)--Subway
Food & Drink Shop--Home (private)
Food & Drink Shop--Food & Drink Shop
Coffee Shop--Bar
Coffee Shop--Office
Bus Station--Home (private)
Bus Station--Bus Station
Office--Subway
Office--Bar
Office--Home (private)
Office--Office
Other Great Outdoors--Home (private)
Subway--Subway--Subway
Subway--Home (private)
Subway--Office
Subway--Train Station
Road--Road
Neighborhood--Home (private)
Neighborhood--Neighborhood
College Academic Building--College Academic Building
Deli / Bodega--Home (private)
Gym / Fitness Center--Bar
Train Station--Subway
Train Station--Office
Train Station--Train Station
Bar--Bar
Bar--Home (private)
CPU times: user 24.2 s, sys: 165 ms, total: 24.4 s
Wall time: 14.6 s


Hmm, this was not super quick, but also not that slow. By caching the ids for sequences that match a pattern, we can speed up calculating support for sequences created by extending it. 

We get too few patterns that are not singletons since the venue categories are too diffused. For instance, there are like 20 different types of restaurant. To get a better look at daily traffic, we should probably cluster similar categories. [TODO]

### Appendix

Code snippets besides the main line. 

In [65]:
# import time

# for i in range(2, 10):
#     st = time.time()
#     q = assemble_support_query([24] * i)
#     conn.execute(q)
#     print(int((time.time() - st) * 1000))